# Data Downloading as CIF file and other property

1. Fetching crystal structures from Materials Project API

In [2]:
# Import required libraries
import csv
import functools
import warnings
import random
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataloader import default_collate
from torch.utils.data.sampler import SubsetRandomSampler

from pymatgen.core.structure import Structure
from pymatgen.core.periodic_table import Element

import pandas as pd
from tqdm import tqdm

##  Fetch Data from Materials Project API
- Environment variable: `MP_API`

In [ ]:
# Import Materials Project API client
from mp_api.client import MPRester

# Get API key (update this based on your setup)
import os
from dotenv import load_dotenv

# This line loads the .env file
load_dotenv() 

# Now you can access the variables using os.getenv()
#api_key = os.getenv('MP_API')
api_key = "your_api_key_here"  # Replace with your actual API key

if not api_key:
    raise ValueError("Please set MP_API environment variable or provide API key directly")

print("API key loaded")

API key loaded


In [ ]:
import os
import time
import pandas as pd
from tqdm import tqdm
from pymatgen.io.cif import CifWriter
from mp_api.client import MPRester

# ---- Config ----
MAX_MATERIALS = 50
CIF_DIR = "data/cif"
os.makedirs(CIF_DIR, exist_ok=True)

# Fields to retrieve
fields = [
    "material_id", "structure", "formula_pretty",
    "formation_energy_per_atom", "energy_above_hull",
    "band_gap", "density", "volume"
]

cif_count = 0
data_list = []

with MPRester(api_key) as mpr:
    # ✅ Use a tuple for ranges, not a dict
    # e.g., energy_above_hull=(0, 0.1) or (None, 0.1)
    all_ids = mpr.materials.summary.search(
        energy_above_hull=(0, 0.1),
        fields=["material_id"]
    )
    material_ids = [doc.material_id for doc in all_ids]
    print(f"🔍 Found {len(material_ids)} candidate materials")

    # Process in batches
    for i in range(0, len(material_ids), MAX_MATERIALS):
        batch_ids = material_ids[i:i + MAX_MATERIALS]
        print(f"\n📦 Processing batch {i // MAX_MATERIALS + 1}: {len(batch_ids)} materials")

        try:
            docs = mpr.materials.summary.search(
                material_ids=batch_ids,
                fields=fields
            )
            print(f"✅ Retrieved {len(docs)} materials")

            for doc in tqdm(docs, desc="Saving CIFs"):
                try:
                    # Require a structure to write CIF
                    structure = getattr(doc, 'structure', None)
                    if structure is None:
                        continue

                    cif_path = os.path.join(CIF_DIR, f"{doc.material_id}.cif")
                    CifWriter(structure).write_file(cif_path)
                    cif_count += 1

                    # Pick a target property
                    target_prop = getattr(doc, 'formation_energy_per_atom', None)
                    if target_prop is None:
                        target_prop = getattr(doc, 'energy_above_hull', None)

                    # If no target prop, drop the CIF we just wrote
                    if target_prop is None:
                        try:
                            os.remove(cif_path)
                            cif_count -= 1
                        except OSError:
                            pass
                        continue

                    data_list.append({
                        "material_id": doc.material_id,
                        "cif_file": f"{doc.material_id}.cif",
                        "formula": getattr(doc, 'formula_pretty', 'Unknown'),
                        "formation_energy_per_atom": getattr(doc, 'formation_energy_per_atom', None),
                        "energy_above_hull": getattr(doc, 'energy_above_hull', None),
                        "band_gap": getattr(doc, 'band_gap', None),
                        "density": getattr(doc, 'density', None),
                        "volume": getattr(doc, 'volume', None),
                        "target": target_prop,
                    })

                except Exception as e:
                    print(f"⚠️ Error processing {getattr(doc, 'material_id', 'unknown')}: {e}")
                    continue

        except Exception as e:
            print(f"❌ API error: {e}")
            time.sleep(5)  # brief backoff

        time.sleep(1)  # Respect API rate limits

# Save metadata
df = pd.DataFrame(data_list)
df.to_csv("cgcnn_dataset.csv", index=False)

print(f"\n✅ Saved {cif_count} CIF files to {CIF_DIR}/")
print(f"✅ Metadata saved to cgcnn_dataset.csv with {len(data_list)} entries")

Retrieving SummaryDoc documents: 100%|█| 10


🔍 Found 103644 candidate materials

📦 Processing batch 1: 50 materials


Retrieving SummaryDoc documents: 100%|█| 50


✅ Retrieved 50 materials


Saving CIFs: 100%|█| 50/50 [00:00<00:00, 25


...


📦 Processing batch 2073: 44 materials


Retrieving SummaryDoc documents: 100%|█| 44


✅ Retrieved 44 materials


Saving CIFs: 100%|█| 44/44 [00:00<00:00, 20



✅ Saved 103644 CIF files to data/cif/
✅ Metadata saved to cgcnn_dataset.csv with 103644 entries
